In [7]:
import math, random
import gym
import numpy as np
import sys
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

In [8]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [9]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
sys.path.append('../')
from common.wrappers import make_atari, wrap_deepmind, wrap_pytorch

In [11]:
env_id = "DemonAttack-v0"
env    = make_atari(env_id)
env    = wrap_deepmind(env)
env    = wrap_pytorch(env)

<h3>Use Cuda</h3>

In [12]:
USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

<h2>Replay Buffer</h2>

In [13]:
from collections import deque

class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
            
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done
    
    def __len__(self):
        return len(self.buffer)

<h2>Epsilon greedy exploration</h2>

In [14]:
epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 500

epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

<h2>Computing Temporal Difference Loss</h2>

In [15]:
def compute_td_loss(batch_size):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)

    state      = Variable(torch.FloatTensor(np.float32(state)))
    next_state = Variable(torch.FloatTensor(np.float32(next_state)), volatile=True)
    action     = Variable(torch.LongTensor(action))
    reward     = Variable(torch.FloatTensor(reward))
    done       = Variable(torch.FloatTensor(done))

    q_values      = model(state)
    next_q_values = model(next_state)

    q_value          = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_value     = next_q_values.max(1)[0]
    expected_q_value = reward + gamma * next_q_value * (1 - done)
    
    loss = (q_value - Variable(expected_q_value.data)).pow(2).mean()
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

In [16]:
def plot(frame_idx, rewards, losses):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.subplot(132)
    plt.title('loss')
    plt.plot(losses)
    plt.show()

<p><hr></p>

<h1>Atari Environment</h1>

In [17]:
class CnnDQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(CnnDQN, self).__init__()
        
        self.input_shape = input_shape
        self.num_actions = num_actions
        
        self.features = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def feature_size(self):
        return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            action  = q_value.max(1)[1].data[0]
        else:
            action = random.randrange(env.action_space.n)
        return action
    
    def predict(self, state):
        state = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), volatile=True)
        q_value = self.forward(state)
        return q_value

In [18]:
model = CnnDQN(env.observation_space.shape, env.action_space.n)

if USE_CUDA:
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=0.00001)

replay_initial = 10000
replay_buffer = ReplayBuffer(100000)

In [19]:
epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 30000

epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

In [22]:
num_frames = 1400000
batch_size = 32
gamma      = 0.99

losses = []
all_rewards = []
episode_reward = 0

q_trajectory = []

state = env.reset()
for frame_idx in range(1, num_frames + 1):
    epsilon = epsilon_by_frame(frame_idx)
    action = model.act(state, epsilon)
    q_values = model.predict(state)
    
    if frame_idx%100 == 0:
        q_trajectory.append([state, q_values])
    
    next_state, reward, done, _ = env.step(action)
    replay_buffer.push(state, action, reward, next_state, done)
    
    state = next_state
    episode_reward += reward
    
    if done:
        state = env.reset()
        all_rewards.append(episode_reward)
        episode_reward = 0
        
    if len(replay_buffer) > replay_initial:
        loss = compute_td_loss(batch_size)
        losses.append(loss.data[0])
        
    if frame_idx % 10000 == 0:
        plot(frame_idx, all_rewards, losses)
        
    print('\rframe_idx: ', frame_idx, end="")

frame_idx:  1854

KeyboardInterrupt: 

In [49]:
import h5py
savepath='test.h5'
with h5py.File(savepath, 'w') as hf:
    hf.create_dataset('all_rewards', data=all_rewards)
    hf.create_dataset('losses', data=losses)
    hf.create_dataset('q_trajectory', data=q_trajectory)

ValueError: could not broadcast input array from shape (84,84) into shape (1)

In [74]:
q_trajectory[0][1].data.cpu().numpy()[0].shape

(6,)

In [5]:
import h5py
savename='test'


def Entropy(P):
    P1 = np.exp(P) / np.sum(np.exp(P))
    return -sum(P1*np.log(P1))

torch.save(model, savename+'_model')

state_trajectory = np.zeros([len(q_trajectory), 84, 84])
Qvalue_trajectory = np.zeros([len(q_trajectory),env.action_space.n])
QEntropy = np.zeros([len(q_trajectory)])


for i in range (len(q_trajectory)):
    state_trajectory[i] = q_trajectory[i][0]
    Qvalue_trajectory[i] = q_trajectory[i][1].data.cpu().numpy()[0]
    QEntropy[i] = Entropy(Qvalue_trajectory[i])


with h5py.File(savename+'_values.h5', 'w') as hf:
    hf.create_dataset('all_rewards', data=all_rewards)
    hf.create_dataset('losses', data=losses)
    hf.create_dataset('state_trajectory', data=state_trajectory)
    hf.create_dataset('Qvalue_trajectory', data=Qvalue_trajectory)
    hf.create_dataset('QEntropy', data=QEntro)

NameError: name 'torch' is not defined

In [16]:
model = torch.load( 'model')

In [86]:
torch.save(model, 'model')

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type CnnDQN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [2]:
import h5py
import numpy as np
savepath='test.h5'
with h5py.File(savepath, 'r') as hf:
    _rewards = np.array(hf.get('all_rewards'))
    _losses = np.array(hf.get('losses'))
    _states = np.array(hf.get('state_trajectory'))
    _q_values = np.array(hf.get('Qvalue_trajectory'))
    _qEntropy = np.array(hf.get('QEntropy'))

In [4]:
_qEntropy

array([1.78761519e+00, 1.78402945e+00, 1.78847349e+00, 1.78789373e+00,
       1.78539329e+00, 1.78862336e+00, 1.78305495e+00, 1.77733849e+00,
       1.78612937e+00, 1.78110106e+00, 1.78076452e+00, 1.78678964e+00,
       1.78045040e+00, 1.78813252e+00, 1.78855167e+00, 1.78883189e+00,
       1.78365271e+00, 1.78682368e+00, 1.77780008e+00, 1.78554424e+00,
       1.78590794e+00, 1.77755622e+00, 1.78641446e+00, 1.78176697e+00,
       1.78755154e+00, 1.78537944e+00, 1.78582925e+00, 1.79074833e+00,
       1.78251339e+00, 1.78067038e+00, 1.78724146e+00, 1.78232590e+00,
       1.76953244e+00, 1.77990010e+00, 1.77752528e+00, 1.78749182e+00,
       1.78207825e+00, 1.78592671e+00, 1.78334994e+00, 1.77959809e+00,
       1.77828988e+00, 1.78892957e+00, 1.78260005e+00, 1.78610177e+00,
       1.77770803e+00, 1.77572496e+00, 1.76776743e+00, 1.78833814e+00,
       1.78244783e+00, 1.78597749e+00, 1.77526509e+00, 1.78276219e+00,
       1.78809928e+00, 1.77861388e+00, 1.78727569e+00, 1.78938369e+00,
      